In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

from source._helpersV4 import logSave,load_parameters

In [65]:
#Get the best model for the centremost sensor
# data_type,centermost = 'smap','D-11' #['E-8','T-3','D-4','E-11','D-11'] smap dataset
data_type,centermost = 'yahoo','60' #[18,60,29,50,25] smap dataset
arch_type = 'best'

p_path = 'experiments/{}_300/{}_p.json'.format(data_type,data_type)
p = load_parameters(p_path)

l_exp = [1,2,3]
l_gen = ['0300']#[str(i).zfill(4) for i in range(10,101,10)]
extend_to = 500
steps_to = 10

l_c = []
df_fname = f'experiments/{data_type}_100/centermost_{centermost}_bestby_lossf.csv'
l_inputs,l_filter,l_loss_f,_ = lists_from_bestby_df(df_fname)

for exp in l_exp:
    for inputs,filter,loss_f in zip(l_inputs,l_filter,l_loss_f):
        for gen in l_gen:
            if any(lf in loss_f for lf in ['ap','f1']):
                diff_type = 'non-diff'
                opt_direction = 'maximize'
            elif any(lf in loss_f for lf in ['mae','ce']):
                diff_type = 'diff'
                opt_direction = 'minimize'
    
            config = {}
            config['fname'] = '{}{}_{}/exp{}_ins{}_{}_{}/{}/ind_{}.npy'.\
                        format(p['experiment_path'],data_type,centermost,exp,inputs,filter,loss_f,arch_type,gen)
            config['gen_fname'] = '{}{}_{}/exp{}_ins{}_{}_{}/{}/ind_'.\
                        format(p['experiment_path'],data_type,centermost,exp,inputs,filter,loss_f,arch_type)
            config['extend_to'] = extend_to
            config['steps_to'] = steps_to
            config['data_type'] = data_type
            config['arch_type'] = arch_type
            l_c += [config]

extend_arch(l_c)

checked all architecture files exists for dataset/yahoo_data/
all architectures for yahoo in folder best were exteded to generation 500


In [57]:
def lists_from_bestby_df(fname):
    df = pd.read_csv(fname)
    return df['inputs'].to_list(),df['filter'].to_list(),df['lossf'].to_list(),df['gen'].to_list()

def check_if_exists(c_l):
    files_missing = []
    for c in c_l:
        if not os.path.exists(c['fname']):
            files_missing += [c['fname']]
    if files_missing == []:
        print('checked all architecture files exists for {}'.format(p['data_path']))
    else:
        print('List of filess missing to get results:')
        for item in files_missing:
            print(item)
        raise ValueError('Correct missing files before calculating project results')

def extend_arch(l_c):
    check_if_exists(l_c)
    for c in l_c:
        fname = c['fname']
        gen_name = c['gen_fname']
        extend = c['extend_to']
        steps = c['steps_to']
        for g in range(300+steps,extend+1,steps):
            new_arch = f'{gen_name}{str(g).zfill(4)}.npy'
            shutil.copy(fname, new_arch)
    print('all architectures for {} in folder {} were exteded to generation {}'.format(c['data_type'],c['arch_type'],extend))
    